In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/Diplomatic2/Implementation")
!ls

 AGNN_1			      elliptic_txs_features.csv   GRL.ipynb
 AGNN_2			      FINAL_RESULTS		  Node2Vec
 AGNN_3			      GAT_1			  node_deg.obj
 DATA			      GAT_2			  node_d.obj
 data_balanced_1000.obj       GAT_2_1			  NoteBooks
 data_balanced_500.obj	      GAT_3			  project
 data_node_deg_features.obj   GAT_FL_1			  requirements.txt
 data.obj		      GAT_RELU_1		  SAGE_1
 DGI			      GAT_TEST			  SAGE_2
 elliptic_txs_classes.csv     GRL_2.ipynb		  SAGE_3
 elliptic_txs_edgelist.csv   'GRL_FINAL (4).ipynb'	  stellar.obj


In [3]:
#SCRIPT 1
#CREATING DATAFRAMES
import pandas as pd
import math

file_classes = 'elliptic_txs_classes.csv'
file_edges = 'elliptic_txs_edgelist.csv'
file_features = 'elliptic_txs_features.csv'

header_classes = ['txID','classes']
header_edges = ['txID_source','txID_target']
header_features = ['txID','time']
#CREATE COLUMN NAMES FOR FEATURES
for i in range(165):
  header_features.append(str(i))

df_classes = pd.read_csv(file_classes,names=header_classes)
df_edges = pd.read_csv(file_edges,names=header_edges)
df_features = pd.read_csv(file_features,names=header_features)

df_classes = df_classes.iloc[1:]
df_edges = df_edges.iloc[1:]

df_classes.reset_index(drop=True, inplace=True)
df_edges.reset_index(drop=True, inplace=True)


In [ ]:
print(df_edges.head())

In [8]:
#SCRIPT 2
# SCRIPT TO CONFIRM THAT CLASSES AND FEATURES CSVs ARE ALLIGNED
counter = 0
#NA DW AUTA POU EINAI OK
for i in range(df_features.shape[0]):
  #FOR CLASSES DATAFRAME
  classs = df_classes.iloc[[i]].txID.astype(str)
  x1  = str(classs).split()
  ##FOR features dataframe
  feature = df_features.iloc[[i]].txID.astype(str)
  x2  = str(feature).split()
  feature_idx = x2[1]
  class_idx = x1[1]


  if class_idx != feature_idx:
    print("FALSE")
    counter = counter + 1

print(counter)

0


In [9]:
#SCRIPT 3
# SCRIPT TO CREATE DICTIONARY OF txIDs to new NODES AND new column of nodes
print(len(df_classes.txID.unique()))
print(len(df_features.txID.unique()))

total = df_features.shape[0]
txID2node = {}
newNodeColumn = []

for i in range(total):

  classs = df_classes.iloc[[i]].txID.astype(str)
  x1  = str(classs).split()
  class_idx = x1[1]

  newNodeColumn.append(i)
  txID2node[class_idx] = i


203769
203769


In [ ]:
#SCRIPT TO ADD NEW COLUMN TO FEATURES AND CLASSES WITH NEW NODE IDs
df_classes['newNodeID'] = newNodeColumn

In [ ]:
#SCRIPT TO ADD NEW COLUMN TO FEATURES AND CLASSES WITH NEW NODE IDs
df_features['newNodeID'] = newNodeColumn

In [12]:
#SCRIPT 4
#SCRIPT TO CREATE SOURCE AND TARGET LIST BASED ON EDGES CSV
total_edges = df_edges.shape[0]

source_list = []
target_list = []

tuple_list = []

counter1 = 0
counter2 = 0

for i in range(total_edges):

  edge = df_edges.iloc[[i]].astype(str)
  x1  = str(edge).split()

  source = x1[3]
  target = x1[4]

  if txID2node.get(source) == None:
    counter1 = counter1 + 1

  if txID2node.get(target) == None:
    counter2 = counter2 + 1

  new_source = txID2node.get(source)
  new_target = txID2node.get(target)

  source_list.append(new_source)
  target_list.append(new_target)

  tp = []
  tp.append(new_source)
  tp.append(new_target)
  t = tuple(tp)
  tuple_list.append(t)

print(counter1,counter2)

0 0


In [14]:
#LOAD DATA OBJECT
import pickle   

fileObj = open('tuple_list.obj', 'wb')
pickle.dump(tuple_list,fileObj)
fileObj.close()


In [ ]:
#SCRIPT 5
#SCRIPT TO NORMALIZE FEATURES

def normalize(collumns,df_test):
  for i in collumns:
    df_test.iloc[:,i]= (df_test.iloc[:,i] - df_test.iloc[:,i].min()) / (df_test.iloc[:,i].max() - df_test.iloc[:,i].min())

print(len(df_features.columns))

column2Norm = []
for i in range(2,len(df_features.columns)-1):
  column2Norm.append(i)

normalize(column2Norm,df_features)

168


In [ ]:
#COPIES OF PREVIOUS CHECKPOINT
features_new = df_features.copy()
classes_new = df_classes.copy()
edges_source_new = source_list.copy()
edges_target_new = target_list.copy()

In [ ]:
#SCRIPT 6
#DELETE UNNESESERY COLUMNS
del features_new["txID"]
del features_new["time"]
del features_new["newNodeID"]
print(features_new.shape)

del classes_new["txID"]
del classes_new["newNodeID"]
print(classes_new.shape)

(203769, 165)
(203769, 1)


In [ ]:
#SCRIPT 7
#ENCODE TARGETS UNKOWKN = 2 LICIT = 1 ILLICIT = 0
##TO BE DELETED
new_list = []
for i in range(len(classes_new)):

  re = classes_new.iloc[[i]].astype(str)
  re2 = str(re).split()
  #print(re2[2])

  if re2[2] == 'unknown':
    two = 2
    new_list.append(two)
  elif re2[2] == '2':
    one = 1
    new_list.append(one)
  else:
    zero = 0
    new_list.append(zero)

print(len(new_list))
print(len(classes_new))

203769
203769


In [ ]:
#SCRIPT 8
#EDW KANOUME SPLIT SE TRAIN VALIDATION TEST 10% 30% 70%
#PRWTO BHMA NA BROUME OLA TA INDEXES LICIT/ILICIT
licit_number = new_list.count(1)
ilicit_number = new_list.count(0)
print(licit_number,ilicit_number)

ilicit_indexes = [index for (index, item) in enumerate(new_list) if item == 0]
licit_indexes = [index for (index, item) in enumerate(new_list) if item == 1]

print(len(licit_indexes))
print(len(ilicit_indexes))
#print(ilicit_indexes[:10])

42019 4545
42019
4545


In [ ]:
#SCRIPT 9
#DEUTERO BHMA SELECT INDEXES FROM LIST ########EINAI OK#######
# gia train 10% ara 50000*0,1 = 5000 2 klaseis kratame to ratio 500 ilicit kai 2000 licit
# gia validation 30% ara 50000*0,3 = 15000 2 klaseis kratame ratio 1500 gia ilicit kai 13500 gia licit
#gia test ola ta ypoloipa
import random

ilicit_indexes2 = ilicit_indexes.copy()
licit_indexes2 = licit_indexes.copy()

#GIA ILICIT TRAIN/VAL/TEST 500 1500 2500
#print(len(ilicit_indexes2))
ilicit_train_indexes = random.sample(ilicit_indexes2, k=500)
#print(len(ilicit_train_indexes))
ilicit_indexes2 = [item for item in ilicit_indexes2 if item not in ilicit_train_indexes]
#print(len(ilicit_indexes2))
ilicit_val_indexes = random.sample(ilicit_indexes2, k=1500)
#print(len(ilicit_val_indexes))
ilicit_indexes2 = [item for item in ilicit_indexes2 if item not in ilicit_val_indexes]
#print(len(ilicit_indexes2))
ilicit_test_indexes = ilicit_indexes2
#print(len(ilicit_test_indexes))

#GIA LICIT TRAIN/VAL/TEST
print(len(licit_indexes2))
licit_train_indexes = random.sample(licit_indexes2, k=500)
print(len(licit_train_indexes))
licit_indexes2 = [item for item in licit_indexes2 if item not in licit_train_indexes]
print(len(licit_indexes2))
licit_val_indexes = random.sample(licit_indexes2, k=14000)
licit_indexes2 = [item for item in licit_indexes2 if item not in licit_val_indexes]
print(len(licit_indexes2))
licit_test_indexes = licit_indexes2
print(len(licit_test_indexes))


42019
500
41519
27519
27519


In [ ]:
#SCRIPT 10
#CREATE TRAIN VALIDATION TEST MASKS with DEBUGGING
#double checked works ok
import torch

#DEBUG SESSION EINAI SWSTO !!!!!
n_nodes = len(classes_new)
debug_mask = torch.zeros(n_nodes, dtype=torch.bool)
debug_mask[ilicit_train_indexes] = True
n = debug_mask.tolist().count(True)
#print(n)
li_idx = debug_mask.tolist()
indexes_tensor = [index for (index, item) in enumerate(li_idx) if item == True]
#print(len(indexes_tensor))
debug_mask[licit_train_indexes] = True
li_idx = debug_mask.tolist()
indexes_tensor = [index for (index, item) in enumerate(li_idx) if item == True]
#print(len(indexes_tensor))

#CREATE ACTUAL MASKS
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
train_mask[ilicit_train_indexes] = True
train_mask[licit_train_indexes] = True
n2 = train_mask.tolist().count(True)
print(n2)

val_mask = torch.zeros(n_nodes, dtype=torch.bool)
val_mask[ilicit_val_indexes] = True
val_mask[licit_val_indexes] = True
n3 = val_mask.tolist().count(True)
print(n3)

test_mask = torch.zeros(n_nodes, dtype=torch.bool)
test_mask[ilicit_test_indexes] = True
test_mask[licit_test_indexes] = True
n3 = test_mask.tolist().count(True)
print(n3)

1000
15500
30064


In [ ]:
#SCRIPT 11
#PYTORCH INSTALL
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

In [ ]:
#SCRIPT 14
import pickle

stellar = open('stellar.obj', 'wb')
#fileObj = open('data_balanced_1000.obj', 'wb')
#fileObj = open('data.obj', 'wb')

pickle.dump(features_new.values,stellar)
pickle.dump(new_list,stellar)
pickle.dump(edges_source_new,stellar)
pickle.dump(edges_target_new,stellar)

stellar.close()

In [ ]:
#GET NODE DEGREEE
import operator as op
#l = [1,1,2,2,3,4,5,7,9]
res = []
for i in range(203769):
  count = op.countOf(edges_target_new,i)
  res.append(count)

print(len(res))


203769


In [ ]:
#SAVE NODE DEGREE
import pickle

node_degree = open('node_d.obj', 'wb')
#fileObj = open('data_balanced_1000.obj', 'wb')
#fileObj = open('data.obj', 'wb')

pickle.dump(res,node_degree)

node_degree.close()

In [ ]:
x_nd = pd.DataFrame(res)

In [ ]:
#SCRIPT 12
#CREATE DATA OBJECT
#DOUBLE CHECKED
import torch
from torch_geometric.data import Data

# CREATE FEATURES TENSOR
x = torch.tensor(features_new.values, dtype=torch.float)
x_node_deg = torch.tensor(x_nd.values, dtype=torch.float)
#CREATE TARGET TENSOR AFTER ENCODING
y = torch.tensor(new_list, dtype=torch.long)

edge_index = torch.tensor([edges_source_new,
                           edges_target_new], dtype=torch.long)


data2 = Data(x=x, y=y, edge_index=edge_index)

In [ ]:
print(data2)

Data(x=[203769, 165], edge_index=[2, 234355], y=[203769])


In [ ]:
import numpy as np

y_new = new_list.copy()
y_new = [i for i in y_new if i != 2]
print(len(y_new))
print(np.unique(y_new))


46564
[0 1]


In [ ]:
#WEIGTHS
#EDW NAFTIAKSW LISTA ME 0,1 IDIA ME THN ANALOGIA TOY TRAIN SET
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight("balanced", classes=np.unique(y_new), y=y_new)
print(class_weights)

[5.12255226 0.55408268]


In [ ]:
#SCRIPT 13
#ADD ATTRIBUTES TO DATA OBJECT
data2.n_id =  torch.arange(data2.num_nodes)
print(data2.n_id)
data2.train_mask = train_mask
data2.val_mask = val_mask
data2.test_mask = test_mask

tensor([     0,      1,      2,  ..., 203766, 203767, 203768])


In [ ]:
print(data2)

Data(x=[203769, 1], edge_index=[2, 234355], y=[203769], n_id=[203769], train_mask=[203769], val_mask=[203769], test_mask=[203769])


In [ ]:
#SCRIPT 14
import pickle

fileObj = open('data_node_deg_features.obj', 'wb')
#fileObj = open('data_balanced_500.obj', 'wb')
#fileObj = open('data_balanced_1000.obj', 'wb')
#fileObj = open('data.obj', 'wb')

pickle.dump(data2,fileObj)
fileObj.close()

In [ ]:
import pickle   
import torch
from torch_geometric.data import Data

fileObj = open('data.obj', 'rb')
data3 = pickle.load(fileObj)
fileObj.close()
print(data3)

Data(x=[203769, 165], edge_index=[2, 234355], y=[203769], n_id=[203769], train_mask=[203769], val_mask=[203769], test_mask=[203769])


In [ ]:
#USE GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
#EDW NA DW ALLES PERIPTWSEIS GIA BATCHES!!!
#Epishs na dw th lista me tous geitones
from torch_geometric.loader import NeighborLoader

train_loader = NeighborLoader(data3,num_neighbors=[-1, -1], shuffle=True,batch_size=203769)

counter = 0
for batch in train_loader:

    print(len(batch.n_id))
    print(len(batch.y))
    print(type(batch))
    print(batch.test_mask)





203769
203769
<class 'torch_geometric.data.data.Data'>
tensor([False, False, False,  ..., False, False, False])


In [ ]:
#EDW ORIZOUME TA MONTELA GIA GRAPHSAGE GAT AGNN
import numpy as np
import pandas as pd
import os
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch_geometric
from torch.nn import Parameter
from torch_geometric.utils.convert import to_networkx
import networkx as nx
import urllib.request
import tarfile
from torch_geometric.nn import GraphSAGE
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import AGNNConv
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import to_networkx

class GraphSAGE(torch.nn.Module):
  """GraphSAGE"""
  def __init__(self, dim_in, dim_h, dim_out):
    super().__init__()
    self.sage1 = SAGEConv(dim_in, dim_h*2)
    self.sage2 = SAGEConv(dim_h*2, dim_h)
    self.sage3 = SAGEConv(dim_h, dim_out)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.01,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = self.sage1(x, edge_index)
    h = torch.relu(h)
    h = F.dropout(h, p=0.5, training=self.training)
    h = self.sage2(h, edge_index)
    h = torch.relu(h)
    h = F.dropout(h, p=0.2, training=self.training)
    h = self.sage3(h, edge_index)
    return h, F.log_softmax(h, dim=1)

class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 64
        self.in_head = 8
        self.out_head = 1
        
        self.conv1 = GATConv(165, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, 2, concat=False,
                             heads=self.out_head, dropout=0.6)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)

    def forward(self,x, edge_index):
        
        # Dropout before the GAT layer is used to avoid overfitting
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x,F.log_softmax(x, dim=1)    

class AGNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = torch.nn.Linear(165, 64)
        self.prop1 = AGNNConv(requires_grad=False)
        self.prop2 = AGNNConv(requires_grad=True)
        self.lin2 = torch.nn.Linear(64, 2)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)

    def forward(self, x, edge_index):
        x = F.dropout(x, training=self.training)
        x = F.relu(self.lin1(x))
        x = self.prop1(x, edge_index)
        x = self.prop2(x, edge_index)
        x = F.dropout(x, training=self.training)
        x = self.lin2(x)
        return x,F.log_softmax(x, dim=1)        

In [ ]:
#SCRIPT 16
#METRICS
!pip install torchmetrics
from torchmetrics.classification import AUROC
from torchmetrics.classification import BinaryPrecision
from torchmetrics.classification import BinaryRecall
from torchmetrics.classification import BinaryConfusionMatrix
from torchmetrics.classification import MulticlassRecall
from torchmetrics.classification import MulticlassPrecision
from torchmetrics.classification import MulticlassAUROC



def conf_matrix(pred_y, y):
  confmat = BinaryConfusionMatrix().to(device)

  return confmat(pred_y, y)

def brecall(pred_y, y):
  #recall = BinaryRecall().to(device)
  recall = MulticlassRecall(num_classes=2, average=None).to(device)

  return recall(pred_y, y)

def bprecision(pred_y, y):
  precision = MulticlassPrecision(num_classes=2, average=None).to(device)

  return precision(pred_y, y)

def auc_roc(pred_y, y):
  aucroc = AUROC(task="binary")
  print(pred_y.shape)
  print(y.shape)
  #aucroc = MulticlassAUROC(num_classes=2, average=None, thresholds=None).to(device)

  return aucroc(pred_y, y)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 14.0 MB/s 


In [ ]:
#TRAIN FUNCTION
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer

    train_loss_total = []
    val_loss_total = []
    best_model = model
    best_val_loss = 1000.0

    model.train()
    for epoch in range(epochs+1):
      total_loss = 0
      acc = 0
      val_loss = 0
      val_acc = 0
      auc_roc_train = 0
      auc_roc_val = 0 
      pre_train = 0 
      pre_val = 0
      recall_train = 0
      recall_val = 0
 
      # Train on batches
      for batch in train_loader:
        optimizer.zero_grad()
        
        _, out = model(batch.x, batch.edge_index)
        loss = criterion(out[batch.train_mask], batch.y[batch.train_mask])
        total_loss += loss

        acc += accuracy(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
        auc_roc_train += auc_roc(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
        pre_train += bprecision(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
        recall_train += brecall(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
        conf_matrix_train = conf_matrix(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
        print(conf_matrix_train)
        loss.backward()
        optimizer.step()

        # Validation
        val_loss += criterion(out[batch.val_mask], batch.y[batch.val_mask])

        val_acc += accuracy(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])
        auc_roc_val += auc_roc(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])
        pre_val += bprecision(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])
        recall_val += brecall(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])
        conf_matrix_val = conf_matrix(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])
        print(conf_matrix_val)

        print("Train accuracy = ", acc/len(train_loader), "Val accuracy = ", val_acc/len(train_loader))
        print("AUC ROC SCORE TRAIN = ",auc_roc_train/len(train_loader),"AUC ROC SCORE VAL = ", auc_roc_val/len(train_loader))
        print("precision SCORE TRAIN = ",pre_train/len(train_loader),"precision SCORE VAL = ", pre_val/len(train_loader))
        print("RECALL SCORE TRAIN = ",recall_train/len(train_loader),"RECALL SCORE VAL = ", recall_val/len(train_loader))

      if(val_loss < best_val_loss):
        best_val_loss = val_loss
        best_model = model
        print("NEW BEST")
      train_loss_total.append(total_loss)
      val_loss_total.append(val_loss)
      
    return best_model, train_loss_total, val_loss_total

def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    model.eval()
    _, out = model(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    auc_roc_acc = auc_roc(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    precision_score = bprecision(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    recall_score = brecall(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    conf_mat = conf_matrix(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])

    return acc, auc_roc_acc, precision_score, recall_score, conf_mat


In [ ]:
#HYPER PARAMETERS DICTIONARYS
#FOR GAT
hp_params_GAT = []
one = {"layers": 2, "loader": 1, "emb_size": 64, "heads":8}#THIS IS FROM
two = {"layers": 2, "loader": 0, "emb_size": 64, "heads":4}
three = {"layers": 2, "loader": 0, "emb_size": 64, "heads":8}
four = {"layers": 2, "loader": 1, "emb_size": 128, "heads":8}
five = {"layers": 3, "loader": 1, "emb_size": 64, "heads":8}
hp_params_GAT.append(one)
hp_params_GAT.append(two)
hp_params_GAT.append(three)
hp_params_GAT.append(four)
hp_params_GAT.append(five)

class GATmodel(torch.nn.Module):
    def __init__(self,layers,emb_size,head_num):
        super(GATmodel, self).__init__()
        self.hid = emb_size
        self.in_head = head_num
        self.out_head = 1
        
        self.conv1 = GATConv(165, self.hid, heads=self.in_head, dropout=0.6)
        self.conv = GATConv(self.hid*self.in_head, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, 2, concat=False, heads=self.out_head, dropout=0.6)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)

    def forward(self,x, edge_index):
        
        # Dropout before the GAT layer is used to avoid overfitting
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x,F.log_softmax(x, dim=1)    




 

In [ ]:
gat_model = GATmodel(2,8,8).to(device)

In [ ]:
data3 = data3.to(device)
print(gat_model)

best, trai_lo, val_lo = train(gat_model,data3,200)

In [ ]:
#TRAIN 2 WITH NO NeEIGHBOR LOADER
weights = [4.0, 1.0]
class_weights = torch.FloatTensor(weights).to(device)

def train2(model, data, epochs,wei):
    criterion = torch.nn.CrossEntropyLoss(weight=wei)
    optimizer = model.optimizer
    

    model.train()
    
    for epoch in range(epochs+1):
      total_loss = 0
      acc = 0
      val_loss = 0
      val_acc = 0
      auc_roc_train = 0
      auc_roc_val = 0 
      pre_train = 0 
      pre_val = 0
      recall_train = 0
      recall_val = 0
 
      # Train on batches
      
      optimizer.zero_grad()
        
      _, out = model(data3.x, data3.edge_index)
      loss = criterion(out[data3.train_mask], data3.y[data3.train_mask])
      total_loss += loss

      acc += accuracy(out[data3.train_mask].argmax(dim=1), data3.y[data3.train_mask])
      auc_roc_train += auc_roc(out[data3.train_mask].argmax(dim=1), data3.y[data3.train_mask])
      pre_train += bprecision(out[data3.train_mask].argmax(dim=1), data3.y[data3.train_mask])
      recall_train += brecall(out[data3.train_mask].argmax(dim=1), data3.y[data3.train_mask])
      conf_matrix_train = conf_matrix(out[data3.train_mask].argmax(dim=1), data3.y[data3.train_mask])
      print(conf_matrix_train)
      loss.backward()
      optimizer.step()

        # Validation
      val_loss += criterion(out[data3.val_mask], data3.y[data3.val_mask])

      val_acc += accuracy(out[data3.val_mask].argmax(dim=1), data3.y[data3.val_mask])
      auc_roc_val += auc_roc(out[data3.val_mask].argmax(dim=1), data3.y[data3.val_mask])
      pre_val += bprecision(out[data3.val_mask].argmax(dim=1), data3.y[data3.val_mask])
      recall_val += brecall(out[data3.val_mask].argmax(dim=1), data3.y[data3.val_mask])
      conf_matrix_val = conf_matrix(out[data3.val_mask].argmax(dim=1), data3.y[data3.val_mask])
      print(conf_matrix_val)

      print("Train accuracy = ", acc, "Val accuracy = ", val_acc)
      print("AUC ROC SCORE TRAIN = ",auc_roc_train,"AUC ROC SCORE VAL = ", auc_roc_val)
      print("precision SCORE TRAIN = ",pre_train,"precision SCORE VAL = ", pre_val)
      print("RECALL SCORE TRAIN = ",recall_train,"RECALL SCORE VAL = ", recall_val)

      # Print metrics every 10 epochs
      if(epoch % 10 == 0):
          print(f'Epoch {epoch:>3} | Train Loss: {total_loss:.3f} '
                f'| Train Acc: {acc*100:>6.2f}% | Val Loss: '
                f'{val_loss:.2f} | Val Acc: '
                f'{val_acc*100:.2f}%')

In [ ]:
#AGNN MODEL TRAIN
agnn = AGNN().to(device)
data3 = data3.to(device)
print(agnn)

best, trai_lo, val_lo = train(agnn,data3,200)

AGNN(
  (lin1): Linear(in_features=165, out_features=64, bias=True)
  (prop1): AGNNConv()
  (prop2): AGNNConv()
  (lin2): Linear(in_features=64, out_features=2, bias=True)
)
torch.Size([2500])
torch.Size([2500])
tensor([[ 194,  306],
        [ 792, 1208]], device='cuda:0')
torch.Size([15000])
torch.Size([15000])
tensor([[ 624,  876],
        [5376, 8124]], device='cuda:0')
Train accuracy =  0.5607999563217163 Val accuracy =  0.5831999778747559
AUC ROC SCORE TRAIN =  tensor(0.4960, device='cuda:0') AUC ROC SCORE VAL =  tensor(0.5089, device='cuda:0')
precision SCORE TRAIN =  tensor([0.1968, 0.7979], device='cuda:0') precision SCORE VAL =  tensor([0.1040, 0.9027], device='cuda:0')
RECALL SCORE TRAIN =  tensor([0.3880, 0.6040], device='cuda:0') RECALL SCORE VAL =  tensor([0.4160, 0.6018], device='cuda:0')
NEW BEST
torch.Size([2500])
torch.Size([2500])
tensor([[   0,  500],
        [   3, 1997]], device='cuda:0')
torch.Size([15000])
torch.Size([15000])
tensor([[    1,  1499],
        [    

In [ ]:
#FUNCTION TO PLOT RESULTS
import matplotlib.pyplot as plt
x1 = trai_lo
plt.plot(x1)
plt.show()

In [ ]:
#TEST SET FOR AGNN
acc_AGNN, auc_roc_acc_AGNN, precision_score_AGNN, recall_score_AGNN, conf_mat_AGNN = test(best,data3)

print(acc_AGNN)
print(auc_roc_acc_AGNN)
print(precision_score_AGNN)
print(recall_score_AGNN)
print(conf_mat_AGNN)

torch.Size([29064])
torch.Size([29064])
0.9124345779418945
tensor(0.5000, device='cuda:0')
tensor([0.0000, 0.9124], device='cuda:0')
tensor([0., 1.], device='cuda:0')
tensor([[    0,  2545],
        [    0, 26519]], device='cuda:0')


In [ ]:
#TRAIN GAT MODEL
gat3 = GAT().to(device)
data3 = data3.to(device)
print(gat3)

# Train Graph Attention Network
train(gat3, data3, 200)

GAT(
  (conv1): GATConv(165, 64, heads=8)
  (conv2): GATConv(512, 2, heads=1)
)
torch.Size([2500])
torch.Size([2500])
tensor([[ 419,   81],
        [1628,  372]], device='cuda:0')
torch.Size([15000])
torch.Size([15000])
tensor([[ 1234,   266],
        [11160,  2340]], device='cuda:0')
Train accuracy =  0.3163999915122986 Val accuracy =  0.23826667666435242
AUC ROC SCORE TRAIN =  tensor(0.5120, device='cuda:0') AUC ROC SCORE VAL =  tensor(0.4980, device='cuda:0')
precision SCORE TRAIN =  tensor([0.2047, 0.8212], device='cuda:0') precision SCORE VAL =  tensor([0.0996, 0.8979], device='cuda:0')
RECALL SCORE TRAIN =  tensor([0.8380, 0.1860], device='cuda:0') RECALL SCORE VAL =  tensor([0.8227, 0.1733], device='cuda:0')
Epoch   0 | Train Loss: 1.258 | Train Acc:  31.64% | Val Loss: 1.33 | Val Acc: 23.83%
torch.Size([2500])
torch.Size([2500])
tensor([[   0,  500],
        [   1, 1999]], device='cuda:0')
torch.Size([15000])
torch.Size([15000])
tensor([[    2,  1498],
        [   17, 13483]], 

In [ ]:
#TEST SET FOR GAT
#APO OTI FENETAI MA8AINEI NA DIALEGEI PANTA THN MAJORITY CLASS ISWS POLLA HEADS
acc_GAT, auc_roc_acc_GAT, precision_score_GAT, recall_score_GAT, conf_mat_GAT = test(gat3,data3)

print(acc_GAT)
print(auc_roc_acc_GAT)
print(precision_score_GAT)
print(recall_score_GAT)
print(conf_mat_GAT)

torch.Size([29064])
torch.Size([29064])
0.9124345779418945
tensor(0.5002, device='cuda:0')
tensor([0.5000, 0.9125], device='cuda:0')
tensor([3.9293e-04, 9.9996e-01], device='cuda:0')
tensor([[    1,  2544],
        [    1, 26518]], device='cuda:0')


In [ ]:
#GRAPH SAGE TRAIN
graphsage1 = GraphSAGE(165, 64, 2).to(device)
data3 = data3.to(device)
print(graphsage1)

# Train GraphSAGE
train(graphsage1, data3, 200)

GraphSAGE(
  (sage1): SAGEConv(165, 128, aggr=mean)
  (sage2): SAGEConv(128, 64, aggr=mean)
  (sage3): SAGEConv(64, 2, aggr=mean)
)
torch.Size([2500])
torch.Size([2500])
tensor([[ 292,  208],
        [1311,  689]], device='cuda:0')
torch.Size([15000])
torch.Size([15000])
tensor([[ 917,  583],
        [8826, 4674]], device='cuda:0')
Train accuracy =  0.39239999651908875 Val accuracy =  0.37273332476615906
AUC ROC SCORE TRAIN =  tensor(0.4643, device='cuda:0') AUC ROC SCORE VAL =  tensor(0.4788, device='cuda:0')
precision SCORE TRAIN =  tensor([0.1822, 0.7681], device='cuda:0') precision SCORE VAL =  tensor([0.0941, 0.8891], device='cuda:0')
RECALL SCORE TRAIN =  tensor([0.5840, 0.3445], device='cuda:0') RECALL SCORE VAL =  tensor([0.6113, 0.3462], device='cuda:0')
Epoch   0 | Train Loss: 0.710 | Train Acc:  39.24% | Val Loss: 0.71 | Val Acc: 37.27%
torch.Size([2500])
torch.Size([2500])
tensor([[   0,  500],
        [   0, 2000]], device='cuda:0')
torch.Size([15000])
torch.Size([15000])


In [ ]:
#TEST SET FOR SAGE
acc_SAGE, auc_roc_acc_SAGE, precision_score_SAGE, recall_score_SAGE, conf_mat_SAGE = test(graphsage1,data3)

print(acc_SAGE)
print(auc_roc_acc_SAGE)
print(precision_score_SAGE)
print(recall_score_SAGE)
print(conf_mat_SAGE)

torch.Size([29064])
torch.Size([29064])
0.95747309923172
tensor(0.8687, device='cuda:0')
tensor([0.7552, 0.9771], device='cuda:0')
tensor([0.7611, 0.9763], device='cuda:0')
tensor([[ 1937,   608],
        [  628, 25891]], device='cuda:0')


In [ ]:
#TRAIN DGI WITH SAGE ENCODER
